In [ ]:
# import some libs
import os
from datetime import datetime
from fnmatch import fnmatch
import pandas as pd
from openpyxl import load_workbook

In [ ]:
df = pd.DataFrame(
    columns=[
        "file",
        "mod_date",
        "sheet",
        "Author",
        "Date completed",
        "Population",
        "Number of ",
        "Number of ",
        "Number of ",
        "Number of ",
    ]
)
df

In [ ]:
path = "data/"
my_filenames = [
    os.path.join(root, name)
    for root, dirs, files in os.walk(path)
    for name in files
    if name.endswith((".xlsx"))
]
my_filenames

In [ ]:
for filename in my_filenames:
    wb = load_workbook(filename)
    sheet_list = [sheet for sheet in wb.sheetnames if fnmatch(sheet, "*Data*")]
sheet_list

In [ ]:
# last modified datex
time_stamp = os.path.getmtime(my_filenames[1])
print(time_stamp)

In [ ]:
# last modified date
mod_date = datetime.fromtimestamp(time_stamp).strftime("%Y-%m-%d")
print(mod_date)

In [ ]:
sheet_title = wb[sheet_list[1]].title
print(sheet_title)

In [ ]:
cell_value = wb[sheet_list[1]]["B2"].value
print(cell_value)

In [ ]:
df1 = df.append(
    {
        "file": my_filenames[1],
        "mod_date": mod_date,
        "sheet": wb[sheet_list[1]].title,
        "Author": wb[sheet_list[1]]["F1"].value,
        "Date completed": wb[sheet_list[1]]["H1"].value,
        "Population": wb[sheet_list[1]]["B2"].value,
        "Number of ": wb[sheet_list[1]]["E2"].value,
        "Number of ": wb[sheet_list[1]]["B3"].value,
        "Number of ": wb[sheet_list[1]]["D3"].value,
        "Number of ": wb[sheet_list[1]]["F3"].value,
    },
    ignore_index=True,
)
df1

In [ ]:
# loop
for filename in my_filenames:
    wb = load_workbook(filename)
    sheet_list = [sheet for sheet in wb.sheetnames if fnmatch(sheet, "*Data*")]
    # last modified date
    mod_date = datetime.fromtimestamp(os.path.getmtime(filename)).strftime("%Y-%m-%d")
    for sheet in sheet_list:
        df2 = df.append(
            {
                "file": filename,
                "mod_date": mod_date,
                "sheet": wb[sheet].title,
                "Author": wb[sheet]["F1"].value,
                "Date completed": wb[sheet]["H1"].value,
                "Population": wb[sheet]["B2"].value,
                "Number of ": wb[sheet]["E2"].value,
                "Number of ": wb[sheet]["B3"].value,
                "Number of ": wb[sheet]["D3"].value,
                "Number of ": wb[sheet]["F3"].value,
            },
            ignore_index=True,
        )
df2

In [ ]:
# excel ingest function

def trashPanda(path):
    """Searches root directory for all excel files, extracts data and returns structured pandas dataframe."""
    my_filenames = [
        os.path.join(root, name)
        for root, dirs, files in os.walk(path)
        for name in files
        if name.endswith((".xlsx"))
    ]
    df = pd.DataFrame(
        columns=[
            "file",
            "mod_date",
            "sheet",
            "Author",
            "Date completed",
            "Population",
            "Number of ",
            "Number of ",
            "Number of ",
            "Number of ",
        ]
    )
    for filename in my_filenames:
        wb = load_workbook(filename)
        sheet_list = [
            sheet for sheet in wb.sheetnames if fnmatch(sheet, "*March 2021*")
        ]
        # last modified date
        mod_date = datetime.fromtimestamp(os.path.getmtime(filename)).strftime(
            "%Y-%m-%d"
        )
        for sheet in sheet_list:
            df = df.append(
                {
                "file": filename,
                "mod_date": mod_date,
                "sheet": wb[sheet].title,
                "Author": wb[sheet]["F1"].value,
                "Date completed": wb[sheet]["H1"].value,
                "Population": wb[sheet]["B2"].value,
                "Number of ": wb[sheet]["E2"].value,
                "Number of ": wb[sheet]["B3"].value,
                "Number of ": wb[sheet]["D3"].value,
                "Number of ": wb[sheet]["F3"].value,
                },
                ignore_index=True,
            )
    return df

In [ ]:
# call function
path = "data/"
df = trashPanda(path)
df.to_csv(r'data/output.csv')